In [1]:
import dropbox
import os
import numpy as np
import pandas as pd
from scipy.stats import t
import math
import collections
from PIL import Image
from dropbox import DropboxOAuth2FlowNoRedirect
import matplotlib.pyplot as plt
import cv2
import pypyodbc
import sys
import traceback
import time

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
from keras.layers import BatchNormalization, concatenate, Input, Dropout, Flatten, Dense
from keras.utils.np_utils import to_categorical  #?
from keras.optimizers import Adam
from keras.metrics import categorical_accuracy

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
cnxn = pypyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                      "Server=028-GUMUS1-SRV;"
                      "Database=KOTON_DB;"
                      "Trusted_Connection=yes;")

In [3]:
access_token='0DQLcsXIojcAAAAAAAgybeB3DOmXKK7bRTLyYwkthbrAiGmpQR4AuGINQkjBXhif'
dbx = dropbox.Dropbox(access_token)

retrieved_image_df = pd.read_excel('C:/Users/Recep/Koton/Data/retrieved_image_list.xlsx', sheet_name='Sheet1')

In [4]:
def get():
    startIndex = 0 #no image for lower indexes
    numberSamples = 92000
    codes = []
    idx = []
    for picture_index in range(startIndex,startIndex + numberSamples):
        code = retrieved_image_df.iloc[picture_index].name
        codes.append(code)
        idx.append(picture_index)
    
    if len(codes)>2000:
        numOfBatches = 100
    else:
        numOfBatches = 1
        
    string_tar = "SELECT [OptionCode] \
      ,[Groups] \
  FROM [KOTON_DB].[dbo].[Ratio_table_gruouped] Where ("
    
    codes_split = np.array_split(codes, numOfBatches)
    indexes_split = np.array_split(idx, numOfBatches)
    
    
    target = []
    for batch in range(numOfBatches):
        _ =[]
        flag2 = True
        for i in codes_split[batch]:
            if flag2:
                _.append("OptionCode='"+i+"'")
                flag2 = False
            else:
                 _.append(" OR OptionCode='"+i+"'")
        query2 = string_tar + ''.join(_) + ')'
        temp = pd.read_sql_query(query2, cnxn, index_col='optioncode')
        target.append(temp)
    target = pd.concat(target)
    
    
    gen_codes = []
    gen_idx = []
    for code, index in zip(codes, idx):
        try:
            target.loc[code]
            gen_codes.append(code)
            gen_idx.append(index)
        except:
            pass
    
    tar = pd.get_dummies(target)
    gen_codes = np.array(gen_codes)
    gen_idx = np.array(gen_idx)
    
    print(tar.shape)
    print(gen_codes.shape)
    print(gen_idx.shape)
    
    gen_codes, gen_codes_test, gen_indexes, gen_indexes_test = train_test_split(gen_codes, gen_idx, 
                                                                                test_size=0.1, random_state=42)
    tar, tar_test, _, _ = train_test_split(tar, gen_idx, test_size=0.1, random_state=42)
    del _
    
    
    bn_model = Xception(include_top=False, weights='imagenet')
    
    print(gen_codes_test.shape)
    valid_pic1 = []
    valid_errorLog = []
    valid_codes = []
    for code, picture_index in zip(gen_codes_test, gen_indexes_test):
        im_index = retrieved_image_df.iloc[picture_index]['image1'+'_index']        
        if (math.isnan(float(im_index))==False):
            filename = str(code).lower()+'_'+'image1'+'_'+str(int(im_index))+'.jpg'
            path = '/Koton_Image_Files/'+filename
            try:
                md, res = dbx.files_download(path)
                temp_pic = cv2.cvtColor(cv2.resize(cv2.imdecode(np.frombuffer(res.content, dtype=np.uint8), -1), (299,299)), cv2.COLOR_BGR2RGB)
                assert temp_pic.shape == (299, 299, 3)
                valid_pic1.append(temp_pic)
                valid_codes.append(code)
            except:
                valid_errorLog.append(path+' Not found')
    print(np.array(valid_pic1).shape)
    valid_pic1 = preprocess_input(np.array(valid_pic1))
    print(valid_errorLog)
    bottleneck_features_validation = bn_model.predict(preprocess_input(np.array(valid_pic1)))
    print(bottleneck_features_validation.shape)
    valid_codes = np.array(valid_codes)
    np.save(open('bottleneck_features_validation_avg.npy', 'wb'), bottleneck_features_validation)
    np.save(open('valid_codes_avg.npy', 'wb'), valid_codes)
    print('Validation saving complated.')

    
    numOfBatches, lastBatch = divmod(tar.shape[0], 16)
    tars = np.array_split(tar, numOfBatches)
    gen_codes = np.array_split(gen_codes, numOfBatches)
    gen_indexes = np.array_split(gen_indexes, numOfBatches)
    
    
    main_pic1 = []
    main_codes = []
    errorLog = []
    for batch in range(numOfBatches):
        pic1 =[]
        print('Batch: '+str(batch+1)+'/'+str(numOfBatches))
        for code, picture_index in zip(gen_codes[batch], gen_indexes[batch]):
            im_index = retrieved_image_df.iloc[picture_index]['image1'+'_index']        
            if (math.isnan(float(im_index))==False):
                filename = str(code).lower()+'_'+'image1'+'_'+str(int(im_index))+'.jpg'
                path = '/Koton_Image_Files/'+filename
                try:
                    md, res = dbx.files_download(path)
                    temp_pic = cv2.cvtColor(cv2.resize(cv2.imdecode(np.frombuffer(res.content, dtype=np.uint8), -1), (299,299)), cv2.COLOR_BGR2RGB)
                    assert temp_pic.shape == (299, 299, 3)
                    pic1.append(temp_pic)
                    main_codes.append(code)
                except:
                    errorLog.append(path+' Not found')
                    batch += 1 #ignore defective batch
        pic1 = preprocess_input(np.array(pic1))
        innp = bn_model.predict(pic1)
#         print('-'+str(innp.shape))
        main_pic1.append(innp)
    
    print(numOfBatches)
    print(np.vstack(main_pic1).shape)
    main_pic1 = np.vstack(main_pic1)
    main_codes = np.array(main_codes)
#     print(main_pic1.shape)
    print(main_codes.shape)
    
    np.save(open('main_pic1_avg.npy', 'wb'), main_pic1)
    np.save(open('main_codes_avg.npy', 'wb'), main_codes)
    print('Train saving complated.')
    
        
    string_tar = "SELECT [OptionCode] \
      ,[Groups] \
  FROM [KOTON_DB].[dbo].[Ratio_table_gruouped] Where ("
    
    
    
    if len(valid_codes)>2000:
        numOfBatches = 100
    else:
        numOfBatches = 1
        
    codes_split = np.array_split(valid_codes, numOfBatches)
        
    target = []
    for batch in range(numOfBatches):
        _ =[]
        flag2 = True
        for i in codes_split[batch]:
            if flag2:
                _.append("OptionCode='"+i+"'")
                flag2 = False
            else:
                 _.append(" OR OptionCode='"+i+"'")
        query2 = string_tar + ''.join(_) + ')'
        temp = pd.read_sql_query(query2, cnxn, index_col='optioncode')
        target.append(temp)
    target = pd.concat(target)
    tar = pd.get_dummies(target)
    
    np.save(open('valid_tar_avg.npy', 'wb'), tar)
    
    string_tar = "SELECT [OptionCode] \
      ,[Groups] \
  FROM [KOTON_DB].[dbo].[Ratio_table_gruouped] Where ("
    
    codes_split = np.array_split(main_codes, numOfBatches)
    
    if len(main_codes)>2000:
        numOfBatches = 100
    else:
        numOfBatches = 1
        
    codes_split = np.array_split(main_codes, numOfBatches)
        
    target = []
    for batch in range(numOfBatches):
        _ =[]
        flag2 = True
        for i in codes_split[batch]:
            if flag2:
                _.append("OptionCode='"+i+"'")
                flag2 = False
            else:
                 _.append(" OR OptionCode='"+i+"'")
        query2 = string_tar + ''.join(_) + ')'
        temp = pd.read_sql_query(query2, cnxn, index_col='optioncode')
        target.append(temp)
    target = pd.concat(target)
    tar = pd.get_dummies(target)
    
    np.save(open('train_tar_avg.npy', 'wb'), tar)
        
    print('Target saving complated.')
   

    
    

    


In [ ]:
get()

(46330, 10)
(46330,)
(46330,)
(4633,)
(4594, 299, 299, 3)
['/Koton_Image_Files/8yak42909ow405_image1_94660.jpg Not found', '/Koton_Image_Files/8yak13725ek001_image1_73965.jpg Not found', '/Koton_Image_Files/8yam45038nw79f_image1_78909.jpg Not found', '/Koton_Image_Files/8yak68920pw999_image1_70732.jpg Not found', '/Koton_Image_Files/7yak47277mdfk6_image1_91028.jpg Not found', '/Koton_Image_Files/8yam42042nw750_image1_78561.jpg Not found', '/Koton_Image_Files/8yam14475ok03i_image1_78808.jpg Not found', '/Koton_Image_Files/8yam12003ok002_image1_80642.jpg Not found', '/Koton_Image_Files/8yak33479ek045_image1_74804.jpg Not found', '/Koton_Image_Files/7yak62778yw000_image1_48865.jpg Not found', '/Koton_Image_Files/8yak88309bm000_image1_72014.jpg Not found', '/Koton_Image_Files/8kak44151fk1cm_image1_59517.jpg Not found', '/Koton_Image_Files/8yam11194lk961_image1_74214.jpg Not found', '/Koton_Image_Files/8kal71020jdbk3_image1_93520.jpg Not found', '/Koton_Image_Files/8yam11675ck100_image1_810

Batch: 397/2606
Batch: 398/2606
Batch: 399/2606
Batch: 400/2606
Batch: 401/2606
Batch: 402/2606
Batch: 403/2606
Batch: 404/2606
Batch: 405/2606
Batch: 406/2606
Batch: 407/2606
Batch: 408/2606
Batch: 409/2606
Batch: 410/2606
Batch: 411/2606
Batch: 412/2606
Batch: 413/2606
Batch: 414/2606
Batch: 415/2606
Batch: 416/2606
Batch: 417/2606
Batch: 418/2606
Batch: 419/2606
Batch: 420/2606
Batch: 421/2606
Batch: 422/2606
Batch: 423/2606
Batch: 424/2606
Batch: 425/2606
Batch: 426/2606
Batch: 427/2606
Batch: 428/2606
Batch: 429/2606
Batch: 430/2606
Batch: 431/2606
Batch: 432/2606
Batch: 433/2606
Batch: 434/2606
Batch: 435/2606
Batch: 436/2606
Batch: 437/2606
Batch: 438/2606
Batch: 439/2606
Batch: 440/2606
Batch: 441/2606
Batch: 442/2606
Batch: 443/2606
Batch: 444/2606
Batch: 445/2606
Batch: 446/2606
Batch: 447/2606
Batch: 448/2606
Batch: 449/2606
Batch: 450/2606
Batch: 451/2606
Batch: 452/2606
Batch: 453/2606
Batch: 454/2606
Batch: 455/2606
Batch: 456/2606
Batch: 457/2606
Batch: 458/2606
Batch: 4

Batch: 910/2606
Batch: 911/2606
Batch: 912/2606
Batch: 913/2606
Batch: 914/2606
Batch: 915/2606
Batch: 916/2606
Batch: 917/2606
Batch: 918/2606
Batch: 919/2606
Batch: 920/2606
Batch: 921/2606
Batch: 922/2606
Batch: 923/2606
Batch: 924/2606
Batch: 925/2606
Batch: 926/2606
Batch: 927/2606
Batch: 928/2606
Batch: 929/2606
Batch: 930/2606
Batch: 931/2606
Batch: 932/2606
Batch: 933/2606
Batch: 934/2606
Batch: 935/2606
Batch: 936/2606
Batch: 937/2606
Batch: 938/2606
Batch: 939/2606
Batch: 940/2606
Batch: 941/2606
Batch: 942/2606
Batch: 943/2606
Batch: 944/2606
Batch: 945/2606
Batch: 946/2606
Batch: 947/2606
Batch: 948/2606
Batch: 949/2606
Batch: 950/2606
Batch: 951/2606
Batch: 952/2606
Batch: 953/2606
Batch: 954/2606
Batch: 955/2606
Batch: 956/2606
Batch: 957/2606
Batch: 958/2606
Batch: 959/2606
Batch: 960/2606
Batch: 961/2606
Batch: 962/2606
Batch: 963/2606
Batch: 964/2606
Batch: 965/2606
Batch: 966/2606
Batch: 967/2606
Batch: 968/2606
Batch: 969/2606
Batch: 970/2606
Batch: 971/2606
Batch: 9

Batch: 1398/2606
Batch: 1399/2606
Batch: 1400/2606
Batch: 1401/2606
Batch: 1402/2606
Batch: 1403/2606
Batch: 1404/2606
Batch: 1405/2606
Batch: 1406/2606
Batch: 1407/2606
Batch: 1408/2606
Batch: 1409/2606
Batch: 1410/2606
Batch: 1411/2606
Batch: 1412/2606
Batch: 1413/2606
Batch: 1414/2606
Batch: 1415/2606
Batch: 1416/2606
Batch: 1417/2606
Batch: 1418/2606
Batch: 1419/2606
Batch: 1420/2606
Batch: 1421/2606
Batch: 1422/2606
Batch: 1423/2606
Batch: 1424/2606
Batch: 1425/2606
Batch: 1426/2606
Batch: 1427/2606
Batch: 1428/2606
Batch: 1429/2606
Batch: 1430/2606
Batch: 1431/2606
Batch: 1432/2606
Batch: 1433/2606
Batch: 1434/2606
Batch: 1435/2606
Batch: 1436/2606
Batch: 1437/2606
Batch: 1438/2606
Batch: 1439/2606
Batch: 1440/2606
Batch: 1441/2606
Batch: 1442/2606
Batch: 1443/2606
Batch: 1444/2606
Batch: 1445/2606
Batch: 1446/2606
Batch: 1447/2606
Batch: 1448/2606
Batch: 1449/2606
Batch: 1450/2606
Batch: 1451/2606
Batch: 1452/2606
Batch: 1453/2606
Batch: 1454/2606
Batch: 1455/2606
Batch: 1456/26

Batch: 1880/2606
Batch: 1881/2606
Batch: 1882/2606
Batch: 1883/2606
Batch: 1884/2606
Batch: 1885/2606
Batch: 1886/2606
Batch: 1887/2606
Batch: 1888/2606
Batch: 1889/2606
Batch: 1890/2606
Batch: 1891/2606
Batch: 1892/2606
Batch: 1893/2606
Batch: 1894/2606
Batch: 1895/2606
Batch: 1896/2606
Batch: 1897/2606
Batch: 1898/2606
Batch: 1899/2606


In [4]:
valid_codes = np.load(open('valid_codes.npy', 'rb'))

In [29]:
    string_tar = "SELECT [OptionCode] \
      ,[NGroup] \
  FROM [KOTON_DB].[dbo].[Ratio_table_gruouped_new] Where ("
    
    
    
    if len(valid_codes)>2000:
        numOfBatches = 100
    else:
        numOfBatches = 1
        
    codes_split = np.array_split(valid_codes, numOfBatches)
        
    target = []
    for batch in range(numOfBatches):
        _ =[]
        flag2 = True
        for i in codes_split[batch]:
            if flag2:
                _.append("OptionCode='"+i+"'")
                flag2 = False
            else:
                 _.append(" OR OptionCode='"+i+"'")
        query2 = string_tar + ''.join(_) + ')'
        temp = pd.read_sql_query(query2, cnxn, index_col='optioncode')
        target.append(temp)
    target = pd.concat(target)
#     tar = pd.get_dummies(target)

In [8]:
valid_codes


array(['8YAK48395PW999', '7YAK32021YK999', '8YKG37570AK002', ...,
       '8KAM44083OK18A', '8YAK56092IW999', '7KBG45001OA480'], dtype='<U14')

In [30]:
target

,ngroup
optioncode,
7YAK62588OW001,0.5-0.55
6YKB16950OK170,0.95-1
8KKG17660OK999,0.85-0.9
7KAK18228GK001,0.1-0.15
8KAK84337FK913,0.35-0.4
8KLK13370MK493,0.7-0.75
7YAK32257UW410,0-0.05
8KAM91161LK023,0.1-0.15
8YKG37570AK002,0.05-0.1


In [20]:
    string_tar = "SELECT [OptionCode] \
      ,[Groups] \
  FROM [KOTON_DB].[dbo].[Ratio_table_gruouped] Where ("
    target = []
    for i in valid_codes:
            if flag2:
                _.append("OptionCode='"+i+"'")
                flag2 = False
            else:
                 _.append(" OR OptionCode='"+i+"'")
    query2 = string_tar + ''.join(_) + ')'
    temp = pd.read_sql_query(query2, cnxn, index_col='optioncode')


In [27]:
target.head()

AttributeError: 'list' object has no attribute 'head'

In [82]:
y_valid = pd.get_dummies(temp)

In [14]:
y_valid = target

In [20]:
y_valid_ = y_valid.reindex(index=pd.DataFrame(valid_codes)[0])
y_valid_.index.name = 'optioncode'

In [32]:
y_valid_

,webrenktext_tr
optioncode,
8YAK48395PW999,Siyah
7YAK32021YK999,Siyah
8YKG37570AK002,Ekru
8KAK10142AA999,Siyah
7KAK18228GK001,Beyaz
8KLK13370MK493,Bordo
7YAK68001PW906,Sarı
7YAM42104KW000,Beyaz
8KAK68000OW399,Mor


In [31]:
import pickle

target.to_pickle('valid_target_20.d')

In [15]:
np.save(open('valid_tar_20_target.npy', 'wb'), target)

In [9]:
train_codes = np.load(open('main_codes.npy', 'rb'))

In [17]:
    if len(train_codes)>2000:
        numOfBatches = 100
    else:
        numOfBatches = 1
        
    train_split = np.array_split(train_codes, numOfBatches)
        
    target = []
    for batch in range(numOfBatches):
        _ =[]
        flag2 = True
        for i in train_split[batch]:
            if flag2:
                _.append("OptionCode='"+i+"'")
                flag2 = False
            else:
                 _.append(" OR OptionCode='"+i+"'")
        query2 = string_tar + ''.join(_) + ')'
        temp = pd.read_sql_query(query2, cnxn, index_col='optioncode')
        target.append(temp)
    target = pd.concat(target)
    tar = pd.get_dummies(target)

In [18]:
target.head()

,ngroup
optioncode,
7KAK63582EW999,0.95-1
8KAK48049CW999,0.2-0.25
8KAL26293JW893,0-0.05
8YKB46347OW023,0.05-0.1
8KKG17219OK001,0.3-0.35


In [116]:
tar.head()

,groups_0-0.1,groups_0.1-0.2,groups_0.2-0.3,groups_0.3-0.4,groups_0.4-0.5,groups_0.5-0.6,groups_0.6-0.7,groups_0.7-0.8,groups_0.8-0.9,groups_0.9-1
optioncode,,,,,,,,,,
8KAK81035AA230,0,1,0,0,0,0,0,0,0,0
8KAM50163AA047,1,0,0,0,0,0,0,0,0,0
8KMG80816AA600,0,1,0,0,0,0,0,0,0,0
8KAL35118AA909,1,0,0,0,0,0,0,0,0,0
8YAK89502GM307,1,0,0,0,0,0,0,0,0,0


In [27]:
# y_train = pd.get_dummies(temp)
y_train = target

In [28]:
y_train_ = y_train.reindex(index=pd.DataFrame(train_codes)[0])
y_train_.index.name = 'optioncode'

In [34]:
y_train_.head()

,webrenktext_tr
optioncode,
8YAM43145LD740,Mavi
8YAL03832MW999,Siyah
7YAM61329LW750,Yeşil
8YAK89502GM307,Pembe
8YAL11841JKF59,Yeşil


In [30]:
np.save(open('train_tar_2_renk.npy', 'wb'), y_train_.values)

In [19]:
target.to_pickle('train_target_20.d')